In [102]:
import os 
import json 
import pandas as pd 
from tqdm import tqdm 
from collections import Counter

In [69]:
dir_path = "../src/data/verifiability"

In [70]:
data = {}
for file in tqdm(os.listdir(dir_path)):
    path = os.path.join(dir_path, file)
    if not path.endswith(".json"):
        continue
    with open(path, "r") as f:
        topic = json.load(f)
    data[file] = topic 

100%|██████████| 626/626 [00:32<00:00, 19.55it/s]


In [71]:
df = pd.DataFrame(data).T
df.shape

(626, 5)

In [72]:
df = pd.DataFrame(data).T
df = df[~df["sourceId"].isna()].copy()
df["origin_id"] = df["sourceId"].apply(lambda s: s.split("-")[0])
df.head()

,source,summaries,sourceId,datasource,dataset,origin_id
4c86c19f87ab9fb1bdfd841b8e31a1b45ab37578409ec82eabebf5bb5644e3ab-perplexity.json,"[[{'id': 0, 'text': 'Here', 'lemma': 'here'}, ...","[{'tokens': [{'id': 0, 'sent_id': 0, 'text': '...",4c86c19f87ab9fb1bdfd841b8e31a1b45ab37578409ec8...,davinci_debate,verifiability,4c86c19f87ab9fb1bdfd841b8e31a1b45ab37578409ec8...
f71880c1058ad17c25bc75e2f80737f5a92429f97db8ef0da9583dc50a28208a-perplexity.json,"[[{'id': 0, 'text': 'Annotation', 'lemma': 'an...","[{'tokens': [{'id': 0, 'sent_id': 0, 'text': '...",f71880c1058ad17c25bc75e2f80737f5a92429f97db8ef...,eli5_live,verifiability,f71880c1058ad17c25bc75e2f80737f5a92429f97db8ef...
49364587115ac904a3b1690fe11cb5ac15b19e1a71e950f3647c97972edda35c-neeva.json,"[[{'id': 0, 'text': '00:00', 'lemma': '00:00'}...","[{'tokens': [{'id': 0, 'sent_id': 0, 'text': '...",49364587115ac904a3b1690fe11cb5ac15b19e1a71e950...,nq_table_without_short_answer,verifiability,49364587115ac904a3b1690fe11cb5ac15b19e1a71e950...
37ebe00cf860c934f5576ef6d7e892fb2244493f5d68c0cedd9ec3f74dc5cda1-perplexity.json,"[[{'id': 0, 'text': 'Site', 'lemma': 'site'}, ...","[{'tokens': [{'id': 0, 'sent_id': 0, 'text': '...",37ebe00cf860c934f5576ef6d7e892fb2244493f5d68c0...,nq_paragraph_without_short_answer,verifiability,37ebe00cf860c934f5576ef6d7e892fb2244493f5d68c0...
03a077f9b62a4694a784fa650c7e51584b78c84fa5edca23298414d2a4669142-neeva.json,"[[{'id': 0, 'text': 'From', 'lemma': 'from'}, ...","[{'tokens': [{'id': 0, 'sent_id': 0, 'text': '...",03a077f9b62a4694a784fa650c7e51584b78c84fa5edca...,nq_paragraph_with_short_answer,verifiability,03a077f9b62a4694a784fa650c7e51584b78c84fa5edca...


In [73]:
interesting_labels = {
    "Citation Completely Supports but Also Refutes Statement",
    "Citation Partially Supports Statement",
    "Citation Provides No Support for Statement"
}

In [74]:
df.iloc[0]["summaries"][0].keys()

dict_keys(['tokens', 'spans', 'qas', 'predicates', 'label', 'summary_id'])

In [75]:
df["len"] = df["summaries"].apply(len)
df["interesting"] = df["summaries"].apply(lambda lst: sum(1 for x in lst if x["label"] in interesting_labels))

In [82]:
df["interesting"].value_counts()

1    219
2    153
3    116
4     66
5     11
6      3
9      1
7      1
Name: interesting, dtype: int64

In [89]:
df.head()

,source,summaries,sourceId,datasource,dataset,origin_id,len,interesting
4c86c19f87ab9fb1bdfd841b8e31a1b45ab37578409ec82eabebf5bb5644e3ab-perplexity.json,"[[{'id': 0, 'text': 'Here', 'lemma': 'here'}, ...","[{'tokens': [{'id': 0, 'sent_id': 0, 'text': '...",4c86c19f87ab9fb1bdfd841b8e31a1b45ab37578409ec8...,davinci_debate,verifiability,4c86c19f87ab9fb1bdfd841b8e31a1b45ab37578409ec8...,4,2
f71880c1058ad17c25bc75e2f80737f5a92429f97db8ef0da9583dc50a28208a-perplexity.json,"[[{'id': 0, 'text': 'Annotation', 'lemma': 'an...","[{'tokens': [{'id': 0, 'sent_id': 0, 'text': '...",f71880c1058ad17c25bc75e2f80737f5a92429f97db8ef...,eli5_live,verifiability,f71880c1058ad17c25bc75e2f80737f5a92429f97db8ef...,4,1
49364587115ac904a3b1690fe11cb5ac15b19e1a71e950f3647c97972edda35c-neeva.json,"[[{'id': 0, 'text': '00:00', 'lemma': '00:00'}...","[{'tokens': [{'id': 0, 'sent_id': 0, 'text': '...",49364587115ac904a3b1690fe11cb5ac15b19e1a71e950...,nq_table_without_short_answer,verifiability,49364587115ac904a3b1690fe11cb5ac15b19e1a71e950...,4,1
37ebe00cf860c934f5576ef6d7e892fb2244493f5d68c0cedd9ec3f74dc5cda1-perplexity.json,"[[{'id': 0, 'text': 'Site', 'lemma': 'site'}, ...","[{'tokens': [{'id': 0, 'sent_id': 0, 'text': '...",37ebe00cf860c934f5576ef6d7e892fb2244493f5d68c0...,nq_paragraph_without_short_answer,verifiability,37ebe00cf860c934f5576ef6d7e892fb2244493f5d68c0...,4,1
03a077f9b62a4694a784fa650c7e51584b78c84fa5edca23298414d2a4669142-neeva.json,"[[{'id': 0, 'text': 'From', 'lemma': 'from'}, ...","[{'tokens': [{'id': 0, 'sent_id': 0, 'text': '...",03a077f9b62a4694a784fa650c7e51584b78c84fa5edca...,nq_paragraph_with_short_answer,verifiability,03a077f9b62a4694a784fa650c7e51584b78c84fa5edca...,5,2


In [98]:
def count_qas(topics):
    count_sum, count_qas = 0, 0
    for topic in topics:
        topic_row = df.loc[f"{topic}.json"]
        count_sum += len(topic_row["summaries"])
        count_qas += sum([len(summary["qas"]) for summary in topic_row["summaries"]])
    return count_sum, count_qas

In [106]:
chosen_topics = df["sourceId"].sample(10).tolist()
chosen_topics

['5493fecb3b55649c32e719484cf5290c9cc8de4b009e012e95005dfee2717acf-perplexity',
 '584551f86b8670a8f8d4ee66ce93c2c067d0f9eba578a5ec8c3050848f1a3556-neeva',
 '946f2a5000033b54e28772f2a26b9116785f4f2e447b8d1d5cfc1d808d22d74b-neeva',
 '009b80f3d3f2422c54b929ca27062e7f34ee4f82a9c2397880d61b0446082273-neeva',
 'ac6abf47dd60e35f6c0c2833afa0a572eba1969210576d29119758c652c2fdd1-perplexity',
 'd1ad3895b68dc7b6ef730336d3e3ca9a5395d291ebfe19d63c1b9921bf561830-perplexity',
 'a135f8aee02665d0c34ffe0d2a639a2e7c50caece89353927cf666694a02e9cc-neeva',
 '62f2ad142d7bdf639b54c36a2c78cb9f5502fce8fc3fd0639c2831a4cab9d3c0-neeva',
 'dc07e5370d5ea81a4a5d0b39a1ea416f49e8b5e975ee8f94eeef9378b5577771-perplexity',
 'a163a2cf218524637e4f88ef868f88d34219822b0edcea866bbee1b0c3308587-neeva']

In [107]:
count_qas(chosen_topics)

(54, 619)

In [112]:
labels = []
for topic in chosen_topics:
    topic_row = df.loc[f"{topic}.json"]
    labels.extend([summary["label"] for summary in topic_row["summaries"]])


In [113]:
Counter(labels)

Counter({'Citation Partially Supports Statement': 14,
         'Citation Completely Supports Statement': 34,
         'Citation Provides No Support for Statement': 2})

In [111]:
chosen_topics = ['608ef52ca6dc3ea33dfd78c2014987a7efd23785a6b87363ffb7c4dbccfe5b75-perplexity',
                 '608ef52ca6dc3ea33dfd78c2014987a7efd23785a6b87363ffb7c4dbccfe5b75-neeva'
 '5f70135447f685d2cfb0b6e41b00f2ac787f50611adfde29dd60e9d10a1df831-perplexity',
 '9befc3bbc7fe5614a69a577553195d42cb7f9468e74d996337a4e67993effa44-perplexity',
 '9befc3bbc7fe5614a69a577553195d42cb7f9468e74d996337a4e67993effa44-neeva',
 'b4f9e9e26e0fd56b0bd5899ce85c75299d6dc5a89c8ce4878bab2c98e752376f-perplexity',
 'b4f9e9e26e0fd56b0bd5899ce85c75299d6dc5a89c8ce4878bab2c98e752376f-neeva',
 'deae1a7412a8eb9c480322daaed12652bdb50d789d09a393a93348350a4febc1-perplexity',
 'ebb3be87f4929f116372109e6e2f166d5ce21d378f61eee6db33e8b306a96ff5-neeva',
 '308bc8aafa9e96c8ef2c67a01024067171ef6ba93e6479c0d2caaabf50a0fc4b-perplexity',
 'affec3fdd0e64a10e1ebf080072586eb4ab38b6f1449502ebf4ba839fd434429-neeva',
 '1dfb92efea40d15b76af074cf37b9b990d50feb10efdaac6636d788f2b0b0e6b-neeva',
 'afa4a5de3563cabb30e68722419064cf4ce932e0cb5024d2bb96b3991f2399f9-perplexity',
'67247a4db1b0891fc049618dee77f53ebf388b9b97aacaa9bfd41864de874853-perplexity']

In [127]:
df[df["origin_id"] == "67247a4db1b0891fc049618dee77f53ebf388b9b97aacaa9bfd41864de874853"]

,source,summaries,sourceId,datasource,dataset,origin_id,len,interesting
67247a4db1b0891fc049618dee77f53ebf388b9b97aacaa9bfd41864de874853-perplexity.json,"[[{'id': 0, 'text': 'Eye', 'lemma': 'eye'}, {'...","[{'tokens': [{'id': 0, 'sent_id': 0, 'text': '...",67247a4db1b0891fc049618dee77f53ebf388b9b97aaca...,wikihow_keywords,verifiability,67247a4db1b0891fc049618dee77f53ebf388b9b97aaca...,4,1


In [ ]:
# next level
['5493fecb3b55649c32e719484cf5290c9cc8de4b009e012e95005dfee2717acf-perplexity',
 '584551f86b8670a8f8d4ee66ce93c2c067d0f9eba578a5ec8c3050848f1a3556-neeva',
 '946f2a5000033b54e28772f2a26b9116785f4f2e447b8d1d5cfc1d808d22d74b-neeva',
 '009b80f3d3f2422c54b929ca27062e7f34ee4f82a9c2397880d61b0446082273-neeva',
 'ac6abf47dd60e35f6c0c2833afa0a572eba1969210576d29119758c652c2fdd1-perplexity',
 'd1ad3895b68dc7b6ef730336d3e3ca9a5395d291ebfe19d63c1b9921bf561830-perplexity',
 'a135f8aee02665d0c34ffe0d2a639a2e7c50caece89353927cf666694a02e9cc-neeva',
 '62f2ad142d7bdf639b54c36a2c78cb9f5502fce8fc3fd0639c2831a4cab9d3c0-neeva',
 'dc07e5370d5ea81a4a5d0b39a1ea416f49e8b5e975ee8f94eeef9378b5577771-perplexity',
 'a163a2cf218524637e4f88ef868f88d34219822b0edcea866bbee1b0c3308587-neeva']